<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:Базовый класс Project в C#, который будет представлять проекты в рамках 
организации</h2>

----

### Вариант задания № 22


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

На основе этого класса разработать 2-3 производных класса, 
демонстрирующих принципы наследования и полиморфизма. В каждом из классов 
должны быть реализованы новые атрибуты и методы, а также переопределены 
некоторые методы базового класса для демонстрации полиморфизма.

#### Дополнительное задание
Добавьте к существующим классам (базовым и производным 3-4 атрибута и метода) используйте в проекте коллекции, делегаты, события.


<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [2]:
#!csharp

using System;
using System.Collections.Generic;
using System.Linq;

// Делегаты и события
public delegate void ProjectStatusChangedHandler(string projectName, string newStatus);
public delegate void ProjectProgressUpdatedHandler(string projectName, double progress);

public class Project
{
    public int ProjectId { get; protected set; }
    public string ProjectName { get; set; }
    public string Status { get; set; } = "Создан";
    public DateTime StartDate { get; set; } = DateTime.Now;
    public double Budget { get; set; }
    public int Priority { get; set; } = 1;
    public string Department { get; set; }
    public DateTime Deadline { get; set; }
    public string ClientName { get; set; }
    public bool IsApproved { get; set; }
    public int TeamSize => _teamMembers.Count;
    
    // Новые атрибуты
    public List<string> Technologies { get; set; } = new List<string>();
    public Dictionary<string, double> Milestones { get; set; } = new Dictionary<string, double>();
    public HashSet<string> Stakeholders { get; set; } = new HashSet<string>();
    public int SuccessScore { get; protected set; }

    protected List<string> _teamMembers = new List<string>();
    
    // События
    public event ProjectStatusChangedHandler StatusChanged;
    public event ProjectProgressUpdatedHandler ProgressUpdated;

    public static int TotalProjects { get; private set; }
    public static List<Project> AllProjects { get; } = new List<Project>();

    public Project() => TotalProjects++;
    
    public Project(int id, string name) : this()
    {
        ProjectId = id;
        ProjectName = name;
        AllProjects.Add(this);
    }

    public virtual void UpdateStatus(string newStatus)
    {
        Status = newStatus;
        StatusChanged?.Invoke(ProjectName, newStatus);
        NotifyTeamMembers($"Статус изменен: {newStatus}");
    }

    protected virtual void OnProgressUpdated(double progress) => ProgressUpdated?.Invoke(ProjectName, progress);

    public virtual void GetProjectDetails()
    {
        Console.WriteLine($"ID: {ProjectId}, Название: {ProjectName}, Статус: {Status}, Команда: {TeamSize} чел.");
    }

    public virtual void AssignTeamMember(string memberName)
    {
        if (!_teamMembers.Contains(memberName))
        {
            _teamMembers.Add(memberName);
            Console.WriteLine($"Добавлен участник {memberName} в {ProjectName}");
        }
    }

    public virtual double CalculateProgress() 
    {
        var progress = Milestones.Any() ? Milestones.Values.Average() : 0;
        OnProgressUpdated(progress);
        return progress;
    }

    // Новые методы
    public void AddTechnology(string technology) => Technologies.Add(technology);
    
    public void AddMilestone(string milestone, double progress) => Milestones[milestone] = progress;
    
    public void AddStakeholder(string stakeholder) => Stakeholders.Add(stakeholder);
    
    public virtual void CalculateSuccessScore() => SuccessScore = (int)(CalculateProgress() * 10 + Priority * 5);

    protected void NotifyTeamMembers(string message)
    {
        _teamMembers.ForEach(member => 
            Console.WriteLine($"Уведомление для {member}: {message}"));
    }

    public virtual bool IsOnTrack() => DateTime.Now <= Deadline;
    
    public int GetDaysRemaining() => (Deadline - DateTime.Now).Days;

    public static void DisplayAllProjects()
    {
        Console.WriteLine($"\n=== ВСЕ ПРОЕКТЫ ({TotalProjects}) ===");
        AllProjects.ForEach(p => { p.GetProjectDetails(); Console.WriteLine("---"); });
    }
}

public class ProductProject : Project
{
    public DateTime ExpectedCompletionDate { get; set; }
    public string ProductVersion { get; set; }
    public int CustomerCount { get; set; }
    public string TargetPlatform { get; set; }
    
    // Новые атрибуты
    public Queue<string> FeatureBacklog { get; } = new Queue<string>();
    public Stack<string> ReleasedVersions { get; } = new Stack<string>();
    public double UserRating { get; set; }

    public ProductProject(int id, string name, DateTime expectedDate) : base(id, name)
    {
        ExpectedCompletionDate = expectedDate;
        Department = "Продуктовая разработка";
        Deadline = expectedDate;
    }

    public override void UpdateStatus(string newStatus)
    {
        Status = $"{newStatus} (Завершение: {ExpectedCompletionDate:dd.MM.yyyy})";
        base.UpdateStatus(Status); // Вызываем базовый метод для вызова события
        NotifyTeamMembers($"Продукт обновлен: {newStatus}");
    }

    public override double CalculateProgress()
    {
        double progress = (DateTime.Now - StartDate).TotalDays / 
                         (ExpectedCompletionDate - StartDate).TotalDays * 100;
        progress = Math.Min(progress, 100);
        OnProgressUpdated(progress);
        return progress;
    }

    // Новые методы
    public void AddFeatureToBacklog(string feature) => FeatureBacklog.Enqueue(feature);
    
    public void DevelopNextFeature()
    {
        if (FeatureBacklog.Count > 0)
        {
            var feature = FeatureBacklog.Dequeue();
            Console.WriteLine($"Разработана функция: {feature}");
            AddMilestone($"Функция: {feature}", 10);
        }
    }
    
    public void ReleaseVersion(string version)
    {
        ReleasedVersions.Push(version);
        Console.WriteLine($"Выпущена версия {version}");
    }
    
    public void DisplayVersionHistory()
    {
        Console.WriteLine($"История версий {ProjectName}: {string.Join(" ← ", ReleasedVersions)}");
    }
}

public class ResearchProject : Project
{
    public double FundingAmount { get; set; }
    public string ResearchArea { get; set; }
    public int PublishedPapers { get; set; }
    
    // Новые атрибуты
    public LinkedList<string> ResearchPapers { get; } = new LinkedList<string>();
    public SortedDictionary<DateTime, string> Experiments { get; } = new SortedDictionary<DateTime, string>();
    public bool HasBreakthrough { get; private set; }

    public ResearchProject(int id, string name, double funding) : base(id, name)
    {
        FundingAmount = funding;
        Department = "Исследования и разработки";
        Deadline = DateTime.Now.AddYears(1);
    }

    public override void AssignTeamMember(string memberName)
    {
        if (memberName.Contains("Dr.") || memberName.Contains("PhD"))
            base.AssignTeamMember(memberName);
    }

    // Новые методы
    public void AddResearchPaper(string paperTitle)
    {
        ResearchPapers.AddLast(paperTitle);
        PublishedPapers++;
        Console.WriteLine($"Добавлена научная работа: {paperTitle}");
    }
    
    public void ScheduleExperiment(DateTime date, string description) => Experiments[date] = description;
    
    public void ConductScheduledExperiments()
    {
        var today = DateTime.Today;
        foreach (var experiment in Experiments.Where(e => e.Key <= today).ToList())
        {
            Console.WriteLine($"Проведен эксперимент: {experiment.Value}");
            Experiments.Remove(experiment.Key);
            AddMilestone($"Эксперимент: {experiment.Value}", 15);
        }
    }
    
    public void MakeBreakthrough()
    {
        HasBreakthrough = true;
        SuccessScore += 50;
        Console.WriteLine($"Прорыв в исследовании {ProjectName}!");
    }
}

public class InfrastructureProject : Project, IRiskyProject
{
    public string InfrastructureType { get; set; }
    
    // Новые атрибуты
    public Dictionary<DateTime, string> MaintenanceLog { get; } = new Dictionary<DateTime, string>();
    public List<string> SecurityProtocols { get; } = new List<string>();
    public int SystemLoad { get; private set; }

    string IRiskyProject.RiskLevel { get; set; } = "Средний";
    List<string> IRiskyProject.IdentifiedRisks { get; } = new List<string>();

    public InfrastructureProject(int id, string name, int executionPeriod) : base(id, name)
    {
        Department = "Инфраструктура";
        Deadline = DateTime.Now.AddDays(executionPeriod);
    }

    // Новые методы
    public void LogMaintenance(string details)
    {
        MaintenanceLog[DateTime.Now] = details;
        Console.WriteLine($"Записано обслуживание: {details}");
    }
    
    public void AddSecurityProtocol(string protocol) => SecurityProtocols.Add(protocol);
    
    public void UpdateSystemLoad(int load)
    {
        SystemLoad = load;
        Console.WriteLine($"Нагрузка системы обновлена: {load}%");
    }
    
    public void DisplayMaintenanceHistory()
    {
        Console.WriteLine($"История обслуживания {ProjectName}:");
        foreach (var log in MaintenanceLog.OrderByDescending(x => x.Key).Take(5))
        {
            Console.WriteLine($"  {log.Key:dd.MM.yyyy}: {log.Value}");
        }
    }

    void IRiskyProject.AssessRisks()
    {
        var riskyProject = (IRiskyProject)this;
        Console.WriteLine($"Оценка рисков {ProjectName}: {riskyProject.RiskLevel}");
        Console.WriteLine($"Выявленные риски: {string.Join(", ", riskyProject.IdentifiedRisks)}");
    }

    void IRiskyProject.AddRisk(string risk)
    {
        var riskyProject = (IRiskyProject)this;
        riskyProject.IdentifiedRisks.Add(risk);
        Console.WriteLine($"Добавлен риск в {ProjectName}: {risk}");
    }
}

public interface IRiskyProject
{
    string RiskLevel { get; set; }
    List<string> IdentifiedRisks { get; }
    void AssessRisks();
    void AddRisk(string risk);
}

// Обработчики событий
public class ProjectMonitor
{
    public void OnStatusChanged(string projectName, string newStatus) =>
        Console.WriteLine($"[МОНИТОР] Проект '{projectName}' сменил статус на: {newStatus}");
    
    public void OnProgressUpdated(string projectName, double progress) =>
        Console.WriteLine($"[МОНИТОР] Прогресс '{projectName}': {progress:F1}%");
}

// Демонстрация
var monitor = new ProjectMonitor();

var productProject = new ProductProject(1, "Мобильное приложение", DateTime.Now.AddMonths(6))
{
    Budget = 50000, ProductVersion = "1.0", CustomerCount = 150, TargetPlatform = "iOS/Android"
};

var researchProject = new ResearchProject(2, "Исследование AI", 50000)
{
    Budget = 75000, ResearchArea = "Искусственный интеллект"
};

var infrastructureProject = new InfrastructureProject(3, "Обновление серверов", 45)
{
    Budget = 100000, InfrastructureType = "Серверная"
};

// Подписка на события
productProject.StatusChanged += monitor.OnStatusChanged;
productProject.ProgressUpdated += monitor.OnProgressUpdated;
researchProject.StatusChanged += monitor.OnStatusChanged;
infrastructureProject.StatusChanged += monitor.OnStatusChanged;

// Использование новых функциональностей
productProject.AddTechnology("React Native");
productProject.AddTechnology("Firebase");
productProject.AddFeatureToBacklog("Push-уведомления");
productProject.AddFeatureToBacklog("Онлайн-оплата");
productProject.DevelopNextFeature();
productProject.ReleaseVersion("1.1");
productProject.ReleaseVersion("1.2");

researchProject.AddResearchPaper("Нейросети для диагностики");
researchProject.ScheduleExperiment(DateTime.Now.AddDays(-1), "Тестирование алгоритма");
researchProject.ScheduleExperiment(DateTime.Now, "Валидация результатов");
researchProject.ConductScheduledExperiments();

infrastructureProject.AddSecurityProtocol("Двухфакторная аутентификация");
infrastructureProject.LogMaintenance("Замена жестких дисков");
infrastructureProject.LogMaintenance("Обновление прошивки");
infrastructureProject.UpdateSystemLoad(65);

var riskyInfra = (IRiskyProject)infrastructureProject;
riskyInfra.AddRisk("Аппаратный сбой");
riskyInfra.AddRisk("Кибератака");

// Полиморфизм
Project[] projects = { productProject, researchProject, infrastructureProject };

Console.WriteLine("=== ДЕМОНСТРАЦИЯ ПОЛИМОРФИЗМА ===");
foreach (var project in projects)
{
    project.UpdateStatus("В процессе");
    project.CalculateProgress();
    project.CalculateSuccessScore();
    Console.WriteLine($"Успешность: {project.SuccessScore} баллов");
    Console.WriteLine("---");
}

Console.WriteLine("\n=== ДОПОЛНИТЕЛЬНАЯ ИНФОРМАЦИЯ ===");
productProject.DisplayVersionHistory();
infrastructureProject.DisplayMaintenanceHistory();
riskyInfra.AssessRisks();

Console.WriteLine($"\nВсего проектов в системе: {Project.TotalProjects}");
Project.DisplayAllProjects();

Разработана функция: Push-уведомления
Выпущена версия 1.1
Выпущена версия 1.2
Добавлена научная работа: Нейросети для диагностики
Проведен эксперимент: Тестирование алгоритма
Записано обслуживание: Замена жестких дисков
Записано обслуживание: Обновление прошивки
Нагрузка системы обновлена: 65%
Добавлен риск в Обновление серверов: Аппаратный сбой
Добавлен риск в Обновление серверов: Кибератака
=== ДЕМОНСТРАЦИЯ ПОЛИМОРФИЗМА ===
[МОНИТОР] Проект 'Мобильное приложение' сменил статус на: В процессе (Завершение: 21.05.2026)
[МОНИТОР] Прогресс 'Мобильное приложение': 0.0%
[МОНИТОР] Прогресс 'Мобильное приложение': 0.0%
Успешность: 5 баллов
---
[МОНИТОР] Проект 'Исследование AI' сменил статус на: В процессе
Успешность: 155 баллов
---
[МОНИТОР] Проект 'Обновление серверов' сменил статус на: В процессе
Успешность: 5 баллов
---

=== ДОПОЛНИТЕЛЬНАЯ ИНФОРМАЦИЯ ===
История версий Мобильное приложение: 1.2 ← 1.1
История обслуживания Обновление серверов:
  21.11.2025: Обновление прошивки
  21.11.2025: